In [96]:
%pylab inline

import mc
import pandas as pd
import project
import scipy.optimize as opt
from importlib import reload

Populating the interactive namespace from numpy and matplotlib


In [405]:
reload(mc)
reload(project)

<module 'project' from '/Users/isaacdrachman/Documents/GitHub/Gradient/project.py'>

In [259]:
options = project.read_data('option_px.csv')

In [325]:
from_day = options.loc[options.Date == '2019-12-06']
selection = from_day.loc[from_day.Strike.isin([7000,7250,7500,7750,8000,8200,8300,8400,8500,8600,8800,9000])]

In [326]:
selection

,Date,Expiration,Strike,CallPut,Bid,Ask,Rate,Spot,Fwd,TTE,Q,Mid
111,2019-12-06,2020-01-17,7000.0,P,6.3,6.9,0.018915,8397.23242,8410.823532,0.113807,0.004705,6.60
131,2019-12-06,2020-01-17,7250.0,P,11.4,12.1,0.018915,8397.23242,8410.823532,0.113807,0.004705,11.75
151,2019-12-06,2020-01-17,7500.0,P,20.5,21.4,0.018915,8397.23242,8410.823532,0.113807,0.004705,20.95
171,2019-12-06,2020-01-17,7750.0,P,36.4,37.5,0.018915,8397.23242,8410.823532,0.113807,0.004705,36.95
191,2019-12-06,2020-01-17,8000.0,P,64.7,65.3,0.018915,8397.23242,8410.823532,0.113807,0.004705,65.00
211,2019-12-06,2020-01-17,8250.0,P,114.9,116.2,0.018915,8397.23242,8410.823532,0.113807,0.004705,115.55
222,2019-12-06,2020-01-17,8400.0,C,172.3,174.8,0.018915,8397.23242,8410.823532,0.113807,0.004705,173.55
230,2019-12-06,2020-01-17,8500.0,C,115.9,117.3,0.018915,8397.23242,8410.823532,0.113807,0.004705,116.60
250,2019-12-06,2020-01-17,8750.0,C,26.5,27.7,0.018915,8397.23242,8410.823532,0.113807,0.004705,27.10
264,2019-12-06,2020-01-17,9000.0,C,3.4,4.0,0.018915,8397.23242,8410.823532,0.113807,0.004705,3.70


In [410]:
mc_options = project.data_to_options(selection)
calibrator = project.Calibrator(mc_options,{'num_paths': 25000, 'num_steps': 100},mc.Heston_gen)

model = {'S0': selection.Spot.iloc[0], 'r': selection.Rate.iloc[0], 'q': selection.Q.iloc[0],
         'nu0': 0.08**2, 'kappa': 4.0, 'theta': 0.16**2, 'xi': 0.30, 'rho': -0.7}
update_keys = ['nu0','kappa','theta','xi','rho']
updates0 = [0.02977745, 1.03335424, 0.06012031, 0.33228585, -0.92116131]

In [387]:
#prices = calibrator.evaluate([],[],model)
prices = calibrator.evaluate(res1['x'],update_keys,model)
for idx,px in enumerate(prices):
    option = selection.iloc[idx]
    print('%d%s %0.2f vs %0.2f/%0.2f'%(option.Strike,option.CallPut,px,option.Bid,option.Ask))

7000P 3.19 vs 6.30/6.90
7250P 8.55 vs 11.40/12.10
7500P 20.95 vs 20.50/21.40
7750P 47.35 vs 36.40/37.50
8000P 97.74 vs 64.70/65.30
8250P 184.83 vs 114.90/116.20
8400C 130.42 vs 172.30/174.80
8500C 90.78 vs 115.90/117.30
8750C 27.09 vs 26.50/27.70
9000C 3.77 vs 3.40/4.00


In [382]:
calibrator.loss(res1['x'],update_keys,model,selection.Mid.values,fixrands=False)

26.97962660206054

In [ ]:
def f(updates):
    return calibrator.loss(updates,update_keys,model,selection.Mid.values)
bounds = [(0.05**2,0.50**2),(0,10),(0.10**2,0.35**2),(0.10,1.50),(-1,0)]
res1 = opt.differential_evolution(f, bounds, maxiter=50, popsize=15, disp=True, updating='deferred', 
                                  workers=-1, polish=True)
res1

differential_evolution step 1: f(x)= 31.0451
differential_evolution step 2: f(x)= 31.0451
differential_evolution step 3: f(x)= 31.0451
differential_evolution step 4: f(x)= 31.0451
differential_evolution step 5: f(x)= 31.0451
differential_evolution step 6: f(x)= 29.8846
differential_evolution step 7: f(x)= 29.8846
differential_evolution step 8: f(x)= 29.8846
differential_evolution step 9: f(x)= 29.8846
differential_evolution step 10: f(x)= 29.5308
differential_evolution step 11: f(x)= 29.5308
differential_evolution step 12: f(x)= 29.5308
differential_evolution step 13: f(x)= 29.5087
differential_evolution step 14: f(x)= 27.5999
differential_evolution step 15: f(x)= 27.5999
differential_evolution step 16: f(x)= 27.5999
differential_evolution step 17: f(x)= 27.5999
differential_evolution step 18: f(x)= 27.019
differential_evolution step 19: f(x)= 27.019
differential_evolution step 20: f(x)= 27.019
differential_evolution step 21: f(x)= 27.019
differential_evolution step 22: f(x)= 27.019
di

In [407]:
def f(updates):
    return calibrator.loss(updates,update_keys,model,selection.Mid.values)
bounds = [(0.05**2,0.50**2),(0,10),(0.10**2,0.35**2),(0.10,1.50),(-1,0)]
cons = [{'type':'ineq', 'fun': lambda x: 2*x[1]*x[2] - x[3]**2}]

res2 = opt.minimize(f, [0.03, 5.0, 0.25**2, 0.50, -0.96], options={'maxiter': 20}, bounds=bounds, constraints=cons)
res2

KeyboardInterrupt: 